In [1]:
!pip install pyarrow

In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import date
from pathlib import Path

In [7]:
print(pd.__version__)
#print(pd.show_versions())

2.3.3


In [6]:
data_path = "../data/usage/finance/lead_lag_analysis/dt=2026-02-24/data.parquet"
table = pq.read_table(data_path)
df = table.to_pandas()
print(f"Dataset size: {df.shape[0]}")

Dataset size: 509


In [10]:
df.head()

,ts_minute_utc,btc_close,btc_volume,btc_high,btc_low,btc_change_pct,ndaq_close,ndaq_volume,ndaq_high,ndaq_low,...,btc_close_lag_5,btc_close_lead_5,btc_volume_lag_5,ndaq_close_lag_5,ndaq_close_lead_5,btc_return_1m,ndaq_return_1m,processed_at_utc,execution_date,dt
0,2026-02-24 01:50:00,64251.37,12.52670,64318.17,64247.05,-0.065139,NaN,NaN,NaN,NaN,...,NaN,64168.82,NaN,NaN,NaN,-0.065155,NaN,2026-02-24 10:40:32.547957,2026-02-24,2026-02-24
1,2026-02-24 01:51:00,64306.47,9.36197,64316.00,64230.00,0.085741,NaN,NaN,NaN,NaN,...,NaN,64182.58,NaN,NaN,NaN,0.085757,NaN,2026-02-24 10:40:32.547957,2026-02-24,2026-02-24
2,2026-02-24 01:52:00,64255.54,8.02958,64339.19,64255.54,-0.079199,NaN,NaN,NaN,NaN,...,NaN,64158.96,NaN,NaN,NaN,-0.079199,NaN,2026-02-24 10:40:32.547957,2026-02-24,2026-02-24
3,2026-02-24 01:53:00,64270.01,1.89922,64278.18,64255.54,0.022504,NaN,NaN,NaN,NaN,...,NaN,64144.90,NaN,NaN,NaN,0.022519,NaN,2026-02-24 10:40:32.547957,2026-02-24,2026-02-24
4,2026-02-24 01:54:00,64239.75,10.08938,64280.84,64214.97,-0.061466,NaN,NaN,NaN,NaN,...,64293.26,64161.68,22.93575,NaN,NaN,-0.047083,NaN,2026-02-24 10:40:32.547957,2026-02-24,2026-02-24


In [12]:
df.isna().sum().sort_values(ascending=False)

ndaq_close           509
ndaq_volume          509
ndaq_close_lag_3     509
ndaq_close_lag_2     509
ndaq_close_lag_1     509
ndaq_close_lead_1    509
ndaq_low             509
ndaq_high            509
ndaq_close_lag_5     509
ndaq_close_lead_4    509
ndaq_return_1m       509
ndaq_close_lead_5    509
ndaq_close_lag_4     509
ndaq_close_lead_3    509
ndaq_close_lead_2    509
btc_close_lead_5       5
btc_volume_lag_5       4
btc_close_lag_5        4
btc_close_lead_4       4
btc_close_lead_3       3
btc_close_lag_4        3
btc_volume_lag_4       3
btc_close_lead_2       2
btc_volume_lag_3       2
btc_close_lag_3        2
btc_close_lead_1       1
btc_volume_lag_2       1
btc_close_lag_2        1
btc_close              0
ts_minute_utc          0
btc_volume             0
btc_high               0
btc_low                0
btc_change_pct         0
btc_close_lag_1        0
btc_volume_lag_1       0
btc_return_1m          0
processed_at_utc       0
execution_date         0
dt                     0


In [4]:
import re
parent_name = Path(data_path).parent.name
m = re.search(r"dt=(\d{4}-\d{2}-\d{2})", parent_name)
date_str = m.group(1) if m else None
print(date_str)
# Compare date_str with current date, today
res = date.fromisoformat(date_str) ==date.today()
print(res)

2026-02-24
True


In [5]:
from typing import Tuple

def get_data_date(filepath: Path) -> Tuple[str | None, bool]:
    is_today = False
    parent_name = filepath.parent.name
    res = re.search(r'dt=(\d{4}-\d{2}-\d{2})', parent_name)
    date_str = res.group(1) if res else None
    if date_str:
        today = date.today()
        is_today = date.fromisoformat(date_str) == today
    return (date_str, is_today)

get_data_date(Path(data_path))

('2026-02-24', True)

In [8]:
print(f"Number of missing values before imputation: {df.isna().sum().sum()}")
print("Imputation with None")
df.where(pd.notna(df), None, inplace=True)
print(f"Number of missing values after imputation: {df.isna().sum().sum()}")
df.head()

Number of missing values before imputation: 4745
Imputation with None
Number of missing values after imputation: 4745


,ts_minute_utc,btc_close,btc_volume,btc_high,btc_low,btc_change_pct,ndaq_close,ndaq_volume,ndaq_high,ndaq_low,...,btc_close_lag_5,btc_close_lead_5,btc_volume_lag_5,ndaq_close_lag_5,ndaq_close_lead_5,btc_return_1m,ndaq_return_1m,processed_at_utc,execution_date,dt
0,2026-02-21 07:22:00,67838.92,0.32025,67838.92,67838.91,0.000000,NaN,NaN,NaN,NaN,...,NaN,67856.01,NaN,NaN,NaN,0.000015,NaN,2026-02-22 00:00:20.024012,2026-02-21,2026-02-21
1,2026-02-21 07:23:00,67838.92,0.34117,67838.92,67838.91,0.000000,NaN,NaN,NaN,NaN,...,NaN,67856.01,NaN,NaN,NaN,0.000000,NaN,2026-02-22 00:00:20.024012,2026-02-21,2026-02-21
2,2026-02-21 07:24:00,67850.02,5.47034,67854.00,67838.91,0.016377,NaN,NaN,NaN,NaN,...,NaN,67856.01,NaN,NaN,NaN,0.016362,NaN,2026-02-22 00:00:20.024012,2026-02-21,2026-02-21
3,2026-02-21 07:25:00,67860.67,0.34730,67860.68,67850.01,0.015696,NaN,NaN,NaN,NaN,...,NaN,67856.01,NaN,NaN,NaN,0.015696,NaN,2026-02-22 00:00:20.024012,2026-02-21,2026-02-21
4,2026-02-21 07:26:00,67856.01,2.63349,67860.68,67856.00,-0.006867,NaN,NaN,NaN,NaN,...,67838.91,67856.01,3.83877,NaN,NaN,-0.006867,NaN,2026-02-22 00:00:20.024012,2026-02-21,2026-02-21


In [13]:
df.loc[0, "ndaq_volume"] = None
df.head(1)

,ts_minute_utc,btc_close,btc_volume,btc_high,btc_low,btc_change_pct,ndaq_close,ndaq_volume,ndaq_high,ndaq_low,...,btc_close_lag_5,btc_close_lead_5,btc_volume_lag_5,ndaq_close_lag_5,ndaq_close_lead_5,btc_return_1m,ndaq_return_1m,processed_at_utc,execution_date,dt
0,2026-02-21 07:22:00,67838.92,0.32025,67838.92,67838.91,0.0,NaN,NaN,NaN,NaN,...,NaN,67856.01,NaN,NaN,NaN,0.000015,NaN,2026-02-22 00:00:20.024012,2026-02-21,2026-02-21


In [ ]:
print(f"Number of missing values before imputation: {df.isna().sum().sum()}")
print("Imputation with None")
df.fillna(None, inplace=True)
print(f"Number of missing values after imputation: {df.isna().sum().sum()}")
df.head()

Number of missing values before imputation: ts_minute_utc          0
btc_close              0
btc_volume             0
btc_high               0
btc_low                0
btc_change_pct         0
ndaq_close           314
ndaq_volume          314
ndaq_high            314
ndaq_low             314
btc_close_lag_1        0
btc_close_lead_1       1
btc_volume_lag_1       0
ndaq_close_lag_1     314
ndaq_close_lead_1    314
btc_close_lag_2        1
btc_close_lead_2       2
btc_volume_lag_2       1
ndaq_close_lag_2     314
ndaq_close_lead_2    314
btc_close_lag_3        2
btc_close_lead_3       3
btc_volume_lag_3       2
ndaq_close_lag_3     314
ndaq_close_lead_3    314
btc_close_lag_4        3
btc_close_lead_4       4
btc_volume_lag_4       3
ndaq_close_lag_4     314
ndaq_close_lead_4    314
btc_close_lag_5        4
btc_close_lead_5       5
btc_volume_lag_5       4
ndaq_close_lag_5     314
ndaq_close_lead_5    314
btc_return_1m          0
ndaq_return_1m       314
processed_at_utc       0
execut

ValueError: Must specify a fill 'value' or 'method'.

In [10]:
cols = df.columns
start = 0
step = 10
for i in range(3):
    end = start + (i + 1) * step
    print(list(cols[start:end]))
    start = end

['ts_minute_utc', 'btc_close', 'btc_volume', 'btc_high', 'btc_low', 'btc_change_pct', 'ndaq_close', 'ndaq_volume', 'ndaq_high', 'ndaq_low']
['btc_close_lag_1', 'btc_close_lead_1', 'btc_volume_lag_1', 'ndaq_close_lag_1', 'ndaq_close_lead_1', 'btc_close_lag_2', 'btc_close_lead_2', 'btc_volume_lag_2', 'ndaq_close_lag_2', 'ndaq_close_lead_2', 'btc_close_lag_3', 'btc_close_lead_3', 'btc_volume_lag_3', 'ndaq_close_lag_3', 'ndaq_close_lead_3', 'btc_close_lag_4', 'btc_close_lead_4', 'btc_volume_lag_4', 'ndaq_close_lag_4', 'ndaq_close_lead_4']
['btc_close_lag_5', 'btc_close_lead_5', 'btc_volume_lag_5', 'ndaq_close_lag_5', 'ndaq_close_lead_5', 'btc_return_1m', 'ndaq_return_1m', 'processed_at_utc', 'execution_date', 'dt']


In [11]:
df.dtypes

ts_minute_utc        datetime64[us]
btc_close                   float64
btc_volume                  float64
btc_high                    float64
btc_low                     float64
btc_change_pct              float64
ndaq_close                  float64
ndaq_volume                 float64
ndaq_high                   float64
ndaq_low                    float64
btc_close_lag_1             float64
btc_close_lead_1            float64
btc_volume_lag_1            float64
ndaq_close_lag_1            float64
ndaq_close_lead_1           float64
btc_close_lag_2             float64
btc_close_lead_2            float64
btc_volume_lag_2            float64
ndaq_close_lag_2            float64
ndaq_close_lead_2           float64
btc_close_lag_3             float64
btc_close_lead_3            float64
btc_volume_lag_3            float64
ndaq_close_lag_3            float64
ndaq_close_lead_3           float64
btc_close_lag_4             float64
btc_close_lead_4            float64
btc_volume_lag_4            